# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [21]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [22]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [24]:

# Carica il dataset Wine Quality White con il parser 'auto' e assicura che pandas sia installato
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True, parser='auto')

X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):

    if hyperparams['use_pca']:
        if hyperparams['pca_standardize']:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)
        pca = PCA(n_components=hyperparams['pca_components'])
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)

    if hyperparams['data_standardize'] and not hyperparams['use_pca']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Aggiunge il termine costante ai dati
    X_train = np.c_[np.ones(X_train.shape[0]), X_train]
    X_val = np.c_[np.ones(X_val.shape[0]), X_val]

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        reg = Ridge(alpha=hyperparams['reg_lambda'])
        reg.fit(X_train, y_train)
        y_pred_val = reg.predict(X_val)
    else:
        reg = LinearRegression()
        reg.fit(X_train, y_train)
        y_pred_val = reg.predict(X_val)

    # Calcolo predizioni
    y_pred_train = reg.predict(X_train)

    # Calcola il MAE
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_val = mean_absolute_error(y_val, y_pred_val)

    return mae_train, mae_val

# Dividi il dataset in training e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dividi il training set in training set effettivo e validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Definisci le possibili configurazioni degli iperparametri
configs = [
    {'use_pca': True, 'pca_standardize': True, 'pca_components': 5, 'data_standardize': False, 'use_regularization': False, 'reg_lambda': 1},
    {'use_pca': False, 'pca_standardize': False, 'pca_components': 5, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 0.1},
    {'use_pca': False, 'pca_standardize': False, 'pca_components': 5, 'data_standardize': True, 'use_regularization': False, 'reg_lambda': 1},
    {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 0.01}
]

# Trova la configurazione di iperparametri migliore
best_config = None
best_mae_val = float('inf')

for config in configs:
    mae_train, mae_val = pipeline(X_train, y_train, X_val, y_val, config)
    if mae_val < best_mae_val:
        best_mae_val = mae_val
        best_config = config

# Riallena il modello sul training set completo
mae_train, mae_val = pipeline(X_train, y_train, X_val, y_val, best_config)

# Calcola il MAE sul test set
_, mae_test = pipeline(X_train, y_train, X_test, y_test, best_config)

# Stampa i risultati
print(f'Migliore configurazione: {best_config}')
print(f'MAE di validation: {best_mae_val}')
print(f'MAE di train: {mae_train}')
print(f'MAE di test: {mae_test}')


Migliore configurazione: {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 0.01}
MAE di validation: 0.5643576064170629
MAE di train: 0.5909976070302998
MAE di test: 0.5949210928868088
